In [1]:
require(data.table)

Loading required package: data.table



In [2]:
channel = "a647"

In [3]:
get_scaling_factor_from_dw_log = function(fpath) {
    stopifnot(file.exists(fpath))
    log_lines = scan(fpath, what="character", sep="\n")
    scaling_line = log_lines[grepl("scaling", log_lines)]    
    return(as.numeric(gsub("scaling: ([0-9]+\\.[0-9]+)", "\\1", scaling_line)))
}

# 60x

In [4]:
root = "/mnt/data/Imaging/202105-Deconwolf/data_210726"

In [5]:
scaling_60x = rbindlist(lapply(1:7, function(series_id, channel, root, magnification, image_type) {
    dpath = file.path(root, sprintf("%s_%s", magnification, image_type), "log")
    fpath = file.path(dpath, sprintf("%s_%03d.tif.log.txt", channel, series_id))
    data.table(
        magnification=magnification,
        image_type=image_type,
        series_id=series_id,
        scaling=get_scaling_factor_from_dw_log(fpath)
    )
}, channel, root, "60x", "dw"))

# 20x

In [6]:
scaling_20x = rbindlist(lapply(scan("/mnt/data/Imaging/202105-Deconwolf/data_210726/20x_dw/matches.txt", what="character"), function(x) {
    transpose(data.table(as.numeric(unlist(strsplit(
        gsub("^20x_dw_a647_([0-9]+)_image_([0-9]+)\\.tif", "\\1,\\2", x), ",")
    ))))
}))
setnames(scaling_20x, c("sid_20x", "sid_60x"))

In [7]:
scaling_20x = scaling_20x[!(sid_20x == 10 & sid_60x == 6)]

In [8]:
root = "/mnt/data/Imaging/202105-Deconwolf/old/C1_20x_c/dw"

In [9]:
scaling_20x$scaling = unlist(lapply(file.path(root, sprintf("%s_%03d.tif.log.txt", channel, scaling_20x$sid_20x)), get_scaling_factor_from_dw_log))

In [10]:
scaling_20x[, sid_20x := NULL]
setnames(scaling_20x, "sid_60x", "series_id")
scaling_20x$magnification = "20x"
scaling_20x$image_type = "dw"

# Output

In [11]:
scaling = rbindlist(list(scaling_20x, scaling_60x), use.names=T)

In [12]:
fwrite(scaling, "../../data/dw_scaling_factors.tsv", sep="\t")